In [ ]:
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
import json
import string
import re
import time
from collections import defaultdict

from array import array
import math
import collections
from numpy import linalg as la
import pandas as pd

In [ ]:
# Read json and store the text of each tweet into a list
def read_json(docs_path):
    with open(docs_path) as fp:
        lines = json.load(fp)

    print("Total number of tweets in the dataset: {}".format(len(lines)))
    return lines

docs_path = 'inputs/dataset_tweets_WHO.txt'
lines = read_json(docs_path)

tweets = []
for tweetId in lines:
    tweets.append(lines[str(tweetId)]["full_text"])

In [ ]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)

url_pattern = re.compile(r'https?://\S+|www\.\S+')

def remove_urls(line):
    return url_pattern.sub(r'', line)

def remove_emojis(line):
    return emoji_pattern.sub(r'', line)

def build_terms(line):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    line = line.encode("ascii", "ignore") # Remove unicode characters
    line = line.decode()
    line = line.lower() ## Transform to lowercase
    line = remove_emojis(line) ## Remove emojis, before tokenizing to delete emojis not separated by space with a word
    line = remove_urls(line) ## Remove urls
    line = line.split() ## Tokenize the text to get a list of terms
    line = [w for w in line if w not in stop_words]  ## eliminate the stopwords
    line = [w for w in line if w[0]!='&' and w[-1]!=';'] ## Remove HTML symbol entity codes
    line = [w.strip(string.punctuation.replace('#', '').replace('@', '')) for w in line] ## Remove punctuation except # and @
    line = [stemmer.stem(w) for w in line if w!=''] ## perform stemming and remove empty words
    return line

In [ ]:
# Execute to see first 10 original and processed tweets
for tweet in tweets[:10]:
    print("Original tweet:\n" + tweet + "\n")
    print("Processed tweet:\n" + str(build_terms(tweet)) + "\n")

PART 2: INDEXING AND EVALUATION

In [ ]:
# 1. Inverted index:
def create_index(lines):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of json tweets
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)
    tweet_index = {}  # dictionary to map tweet ids (starting from 0) to their info
    for tweetId in lines:  # lines contain all tweets, whith the id as key
        full_tweet = lines[str(tweetId)]
        
        tweet_id = full_tweet["id"] # id 
        tweet = full_tweet["full_text"] # Tweet
        username = full_tweet["user"]["screen_name"] # Username
        date = full_tweet["created_at"] # Date
        hashtags = full_tweet["entities"]["hashtags"] # Hashtags
        likes = full_tweet["favorite_count"] # Likes
        retweets = full_tweet["retweet_count"] # Retweets
        url = f"https://twitter.com/{username}/status/{tweet_id}" # Url
        terms = build_terms(tweet)
        
        # Store tweet info in the dictonary to retrieve it faster when searching
        tweet_index[tweetId] = {}
        tweet_index[tweetId]["tweet"] = tweet
        tweet_index[tweetId]["username"] = username
        tweet_index[tweetId]["date"] = date
        tweet_index[tweetId]["hashtags"] = hashtags
        tweet_index[tweetId]["likes"] = likes
        tweet_index[tweetId]["retweets"] = retweets
        tweet_index[tweetId]["url"] = url
        ## ===============================================================        
        ## create the index for the current page and store it in current_page_index (current_page_index)
        ## current_page_index ==> { ‘term1’: [current_doc, [list of positions]], ...,‘term_n’: [current_doc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ##"web retrieval information retrieval":

        ## current_page_index ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}

        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================

        current_page_index = {}

        for position, term in enumerate(terms): # Loop over all terms
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                current_page_index[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[tweetId, array('I',[position])] #'I' indicates unsigned int (int in Python)
            
        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
        
        ## END CODE                    
                    
    return index, tweet_index


In [ ]:
# Index creation and test
start_time = time.time()
index, tweet_index = create_index(lines)
print("Total time to create the index: {} seconds".format(np.round(time.time() - start_time, 2)))

print("Index results for the term 'researcher': {}\n".format(index['international']))
print("First 10 Index results for the term 'research': \n{}".format(index['intern'][:10]))

In [ ]:
def search(query, index):
    """
    The output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            # docs = docs Union term_docs
            docs = docs.union(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    return docs

In [ ]:
# Test by inserting a query
print("Insert your query (i.e.: Computer Science):\n")
query = input()
docs = search(query, index)
top = 10

print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))
for d_id in docs[:top]:
    print("page_id= {} - tweet: {}".format(d_id, tweet_index[d_id]["tweet"]))

In [ ]:
# 2. Make a proposal of 5 queries that will be used to evaluate your search engine
queries = [
    "covid pandemic",
    "international disaster",
    "ritmo de vacunacion",
    "percentage de hospitalizados",
    "mental health"
]
top = 3 # Number of tweets to show
for query in queries:
    docs = search(query, index) # obtain the tweets resulting of such query
    print("\n======================\nSample of {} results out of {} for the query {}:\n".format(top, len(docs), query))
    for d_id in docs[:top]:
        print("page_id= {} - tweet: {}".format(d_id, tweet_index[d_id]["tweet"]))

In [ ]:
# 3. Apply a TF-IDF ranking to your results.
def create_index_tfidf(lines, num_documents):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of tweets
    num_documents -- total number of tweets
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """

    index = defaultdict(list)
    tf = defaultdict(list)  # term frequencies of terms in documents
    df = defaultdict(int)  # document frequencies of terms in the corpus
    tweet_index = defaultdict(str)
    idf = defaultdict(float)

    for tweetId in lines:
        full_tweet = lines[str(tweetId)]
        
        tweet_id = full_tweet["id"] # id 
        tweet = full_tweet["full_text"] # Tweet
        username = full_tweet["user"]["screen_name"] # Username
        date = full_tweet["created_at"] # Date
        hashtags = full_tweet["entities"]["hashtags"] # Hashtags
        likes = full_tweet["favorite_count"] # Likes
        retweets = full_tweet["retweet_count"] # Retweets
        url = f"https://twitter.com/{username}/status/{tweet_id}" # Url
        terms = build_terms(tweet)
        
        # Store tweet info in the dictonary to retrieve it faster when searching
        tweet_index[tweetId] = {}
        tweet_index[tweetId]["tweet"] = tweet
        tweet_index[tweetId]["username"] = username
        tweet_index[tweetId]["date"] = date
        tweet_index[tweetId]["hashtags"] = hashtags
        tweet_index[tweetId]["likes"] = likes
        tweet_index[tweetId]["retweets"] = retweets
        tweet_index[tweetId]["url"] = url

        current_page_index = {}

        for position, term in enumerate(terms):
            try:
                # if the term is already in the dict append the position to the corresponding list
                current_page_index[term][1].append(position)
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[tweetId, array('I',[position])]

        # Normalize term frequencies
        norm = 0
        for term, posting in current_page_index.items():
            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)

        # Calculate the tf and df weights
        for term, posting in current_page_index.items():
            tf[term].append(np.round(len(posting[1])/norm,4))
            df[term] += 1 # increment DF for current term

        # Merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)

        # Compute IDF following the formula (3) above. HINT: use np.log
        for term in df:
            idf[term] = np.round(np.log(float(num_documents/df[term])), 4)

    return index, tf, df, idf, tweet_index


In [ ]:
start_time = time.time()
num_documents = len(lines)
index, tf, df, idf, title_index = create_index_tfidf(lines, num_documents)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time, 2)))


In [ ]:

def rank_documents(terms, docs, index, idf, tf, tweet_index):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    title_index -- mapping between page id and page title
    
    Returns:
    Print the list of ranked documents
    """

    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query. 

    query_norm = sum(query_terms_count.values())
    for termIndex, term in enumerate(terms):  # termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf[term] 

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):           
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]

    # Calculate the score of each doc 
    # compute the cosine similarity between queyVector and each docVector:
    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    
    result_docs = [x[1] for x in doc_scores]

    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)

    return result_docs


In [ ]:

def search_tf_idf(query, index):
    """
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"                        
            term_docs=[posting[0] for posting in index[term]]
            
            # docs = docs Union term_docs
            docs = docs.union(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs = rank_documents(query, docs, index, idf, tf, tweet_index)
    return ranked_docs

In [ ]:
print("Insert your query (i.e.: Computer Science):\n")
query = input()
ranked_docs = search_tf_idf(query, index)
top = 10

print("\n======================\nTop {} results out of {} for the searched query:\n".format(top, len(ranked_docs)))
for d_id in ranked_docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, tweet_index[d_id]))

EVALUATION

In [ ]:
search_results = pd.read_csv("inputs/test_predictions.csv")
search_results.head()
print("The ground truth of our dataset is composed of {} Relevance Levels: {}" .format(len(search_results["y_true"].unique()), sorted(search_results["y_true"].unique())))
search_results["bin_y_true"] = np.where(search_results["y_true"] >= 2.0, 1, 0)
search_results.head()
search_results["bin_y_true"] = search_results["y_true"].apply(lambda y: 1 if y >=2 else 0)
search_results.head()

In [ ]:
def precision_at_k(y_true, y_score, k=10):
    '''    
    Parameters
    ----------
    y_true: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.
    
    Returns
    -------
    precision @k : float
    
    '''    
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    relevant = sum(y_true == 1)
    return float(relevant) / k

In [ ]:
# Check for query 0

current_query = 0
current_query_res = search_results[search_results["q_id"] == current_query] 
k=5

print("==> Precision@{}: {}\n".format(k,
                                precision_at_k(current_query_res["bin_y_true"], current_query_res["predicted_relevance"], k)))
print("\nCheck on the dataset sorted by score:\n")
current_query_res.sort_values("predicted_relevance", ascending=False).head(k)
k=3
print("==> Precision@{}: {}\n".format(k,
                                precision_at_k(current_query_res["bin_y_true"], current_query_res["predicted_relevance"], k)))
print("\nCheck on the dataset sorted by score:\n")
current_query_res.sort_values("predicted_relevance", ascending=False).head(k)
k=10
print("==> Precision@{}: {}\n".format(k,
                                precision_at_k(current_query_res["bin_y_true"], current_query_res["predicted_relevance"], k)))
print("\nCheck on the dataset sorted by score:\n")
current_query_res.sort_values("predicted_relevance", ascending=False).head(k)



In [ ]:
def avg_precision_at_k(y_true, y_score, k=10):
    
    '''
    Parameters
    ----------
    y_true: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.
    
    Returns
    -------
    average precision @k : float
    '''
    gtp = np.sum(y_true == 1) 
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])            
    ## if all docs are not relevant
    if gtp==0:
        return 0
    n_relevant_at_i = 0
    prec_at_i = 0
    for i in range(len(y_true)):
        if y_true[i] == 1:
            n_relevant_at_i +=1
            prec_at_i += n_relevant_at_i/(i+1)
    return prec_at_i/gtp

In [ ]:
print(avg_precision_at_k(np.array(current_query_res["bin_y_true"]), np.array(current_query_res["predicted_relevance"]), 150))
from sklearn.metrics import average_precision_score
k = 150
temp = current_query_res.sort_values("predicted_relevance", ascending=False).head(k)
average_precision_score(np.array(temp["bin_y_true"]), np.array(temp["predicted_relevance"][:k]))

In [ ]:
def map_at_k(search_res, k=10):
    '''
    Parameters
    ----------
    search_res: search results dataset containing:
        q_id: query id.
        doc_id: document id.
        predicted_relevance: relevance predicted through LightGBM.
        y_true: actual score of the document for the query (ground truth).
    
    Returns
    -------
    mean average precision @k : float
    '''
    avp = []
    for q in search_res["q_id"].unique(): #loop over all query id
        curr_data = search_res[search_res["q_id"] == q]  # select data for current query
        avp.append(avg_precision_at_k(np.array(curr_data["bin_y_true"]), np.array(curr_data["score"]),k)) #append average precision for current query
    return np.sum(avp)/len(avp) # return mean average precision

In [ ]:
map_k,avp = map_at_k(search_results, 10)
map_k

In [ ]:
def rr_at_k(y_true, y_score, k=10):
    '''
    Parameters
    ----------
    y_true: Ground truth (true relevance labels).
    y_score: Predicted scores.
    k : number of doc to consider.
    
    Returns
    -------
    Reciprocal Rank for qurrent query
    '''

    order = np.argsort(y_score)[::-1] # get the list of indexes of the predicted score sorted in descending order.
    y_true = np.take(y_true,order[:k]) # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.
    if np.sum(y_true) == 0: # if there are not relevant doument return 0
        return 0
    return 1/(np.argmax(y_true == 1)+1) # hint: to get the position of the first relevant document use "np.argmax"

In [ ]:
y_true = np.array([0,1,0,1,1])
score = np.array([0.9, 0.5, 0.6, 0.7, 0.2])
rr_at_k(y_true, score,5)

In [ ]:
current_query = 8
current_query_res = search_results[search_results["q_id"] == current_query] 
current_query_res.sort_values("score", ascending=False).head(10)

In [ ]:
labels = np.array(search_results[search_results['q_id'] == 8]["bin_y_true"])
scores = np.array(search_results[search_results['q_id'] == 8]["score"])
np.round(rr_at_k(labels, scores, 10),4)

In [ ]:
mrr = {}
for k in [3,5,10]:
    RRs = []
    for q in search_results['q_id'].unique(): # loop over all query ids
        labels = np.array(search_results[search_results['q_id'] == q]["bin_y_true"]) # get labels for current query
        scores = np.array(search_results[search_results['q_id'] == q]["score"]) # get predicted score for current query
        RRs.append(rr_at_k(labels, scores, k)) # append RR for current query
    mrr[k] = np.round(float(sum(RRs)/len(RRs)),4) # Mean RR at current k

In [ ]:
def dcg_at_k(y_true, y_score,  k=10):
    order = np.argsort(y_score)[::-1] # get the list of indexes of the predicted score sorted in descending order.
    y_true = np.take(y_true, order[:k]) # sort the actual relevance label of the documents based on predicted score(hint: np.take) and take first k.
    gain = 2 ** y_true - 1 # Compute gain (use formula 7 above)
    discounts = np.log2(np.arange(len(y_true)) + 2) # Compute denominator
    return np.sum(gain / discounts) #return dcg@k


def ndcg_at_k(y_true, y_score, k=10):    
    dcg_max = dcg_at_k(y_true, y_true, k)
    if not dcg_max:
        return 0
    return np.round(dcg_at_k(y_true, y_score, k) / dcg_max,4)

In [ ]:
q_id = 0
k = 10
labels = np.array(search_results[search_results['q_id'] == q_id]["y_true"])
scores = np.array(search_results[search_results['q_id'] == q_id]["score"])
ndcg_k = np.round(ndcg_at_k(labels, scores, k),4)
print("ndcg@{} for query with q_id={}: {}".format(k,q_id,ndcg_k))


In [ ]:
ndcgs = []
k=10
for q in search_results['q_id'].unique():
    labels = np.array(search_results[search_results['q_id'] == q]["y_true"])
    scores = np.array(search_results[search_results['q_id'] == q]["score"])
    ndcgs.append(np.round(ndcg_at_k(labels, scores, k),4))

avg_ndcg = np.round(float(sum(ndcgs)/len(ndcgs)),4)
print("Average ndcg@{}: {}".format(k,avg_ndcg))